In [8]:
##### connect to the elasticsearch server
import json
from pprint import pprint
import os
import time
import pandas as pd 
from datetime import datetime
import pathlib

from dotenv import load_dotenv
from elasticsearch import Elasticsearch

import minio
from minio.error import S3Error
from minio.commonconfig import ENABLED
from minio.versioningconfig import VersioningConfig


from DataProfile_JSON_db import *
from minio_utils import *

##### input args
path_to_main_input = "./examples/dummy_data"    
minio_credential = "credentials.macstudio.json"

class RDS:
    def __init__(self, username, password, minio_credential,  verbose = False):
        ##### connect to elastic search database
        self.username = username
        self.password = password
        self.es = Elasticsearch(
            "http://localhost:9200", # deployed locally, no cloud
            basic_auth=(username, password)) 
        client_info = self.es.info()
        tmp = self.es.cat.indices(index='*', h='index', s='index:asc', format='json')
        self.all_indices = [index['index'] for index in tmp if index['index'][0] != "."] # not show hidden indices
        if self.verbose:
            print('Connected to Elasticsearch!')
            pprint(client_info.body)
            
        ##### load minio server information
        self.minio_credential = minio_credential
        with open(self.minio_credential, 'r') as file:
            keys = json.load(file)
        minio_client = minio.Minio(
            keys['endpoint'],
            access_key=keys['access_key'],
            secret_key=keys['secret_key'],
            secure=keys['secure'])
        self.verbose = verbose
        self.minio_client = minio_client
        self.buckets = minio_client.list_buckets()
        
    def es_create_index(self, index_name, metadata_profile):
        self.es.indices.create(index = index_name,  mappings = metadata_profile)

    def es_insert_document(self, document, index):
        return self.es.index(index = index, body=document)
    
    def minio_create_bucket(self):
        
    def minio_upload_file(self, 
                          bucket_name, 
                          object_name, 
                          file_to_upload, 
                          metadata):
        """
        Upload a file to a MinIO bucket with specified metadata.

        Parameters:
        - bucket_name (str): The name of the target bucket.
        - object_name (str): The object name in the bucket.
        - file_to_upload (str): The local path to the file to be uploaded.
        - metadata (dict): A dictionary of metadata to attach to the object.
        - minio_credentials (str): Path to the JSON file with MinIO credentials.

        Returns:
        - bool: True if the file was uploaded successfully, False otherwise.
        """
        with open(self.minio_credential, 'r') as file:
            keys = json.load(file)
        
        minio_client = minio.Minio(
            endpoint="localhost:9000",
            access_key=keys["accessKey"],
            secret_key=keys["secretKey"],
            secure=False 
        )
        
        try:
            with open(file_to_upload, 'rb') as file_data:
                file_stat = os.stat(file_to_upload)
                minio_client.put_object(
                    bucket_name=bucket_name,
                    object_name=object_name,
                    data=file_data,
                    length=file_stat.st_size,
                    metadata=metadata
                )
            if self.verbose:
                print(f"File '{object_name}' uploaded successfully with metadata.")
            return True
        except S3Error as e:
            print(f"Error uploading file: {e}")
            return False
        
    def add_data(self, 
                 bucket_name, 
                 object_name, 
                 file_to_upload, 
                 metadata):

my_es = RDS(username = "elastic", 
            password = "genov4", 
            minio_credential = minio_credential, 
            verbose = False)

In [9]:
my_es